## Pauli Transfer Matrix extraction
$\def\bra#1{\mathinner{\left\langle{#1}\right|}}\def\ket#1{\mathinner{\left|{#1}\right\rangle}}$

In [1]:
import numpy as np; pi = np.pi
import scipy.io as spio
from qutip import *

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# Figures should have a width of a 8.6 cm or 3 3/8 in, the width of a single manuscript column.
golden_mean = (np.sqrt(5)-1.0)/2.0 # Aesthetic ratio
fig_width = 3+3/8 # width in inches
fig_height = fig_width*golden_mean # height in inches (0.75 is the standard height to width ratio in Python)
fig_size = [fig_width, fig_height]
mpl.rcParams.update({
    'axes.labelsize' : 12,
    'font.size' : 12,
    'legend.fontsize' : 8,
    'xtick.labelsize' : 10,
    'ytick.labelsize' : 10,
    'ytick.minor.pad' : -0.5,
    'ytick.minor.pad' : -0.5,
    'ytick.major.size' : 1,
    'ytick.minor.size' : 1,
    'ytick.major.width' : .5,
    'ytick.minor.width' : .5,
    'xtick.major.size' : 1,
    'xtick.minor.size' : 1,
    'xtick.major.width' : .5,
    'xtick.minor.width' : .5,
    'xtick.major.pad' : 1.5,
    'xtick.minor.pad' : 1.5,
    'text.usetex' : True,
    'figure.dpi' : 100,
})

## Parameters

In [2]:
num_lvl = 20 # number of levels
a = destroy(num_lvl) # annihilation operator
K = 1 # kerr amplitude
G = 4*K # two photon pump amplitude
alpha = np.sqrt(G/K) # amplitude

# cat states
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

# computational basis
up = (cat_plus + cat_minus)/np.sqrt(2)
down = (cat_plus - cat_minus)/np.sqrt(2)

# Identity
I = up*up.dag() + down*down.dag()

# sigma-z in computational basis
sigma_z = up*up.dag() - down*down.dag()

# sigma-x in computational basis
sigma_x = up*down.dag() + down*up.dag()

# sigma-y in computational basis
sigma_y = 1j*(-up*down.dag() + down*up.dag())

# Array with Pauli matrices
P = [I, sigma_x, sigma_y, sigma_z]

## $R_z(\phi)$-gate

In [3]:
# gate time
T_g = 2/K

# initial state
psi0 = (up+down).unit()

# single photon pump amplitude
def E(t,args):
    phi = args['phi']
    return np.pi*phi/(8*T_g*alpha)*np.sin(np.pi*t/T_g)

def rz(phi):
    return (-1j*phi/2*sigma_z).expm()

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = a.dag() + a
H_tot = [H0,[H1,E]]

# time array
tlist = np.linspace(0,T_g,200)

# angle array
phi_list = np.linspace(-np.pi,np.pi,20)

### quantum map

A quantum map $\Lambda$ is the following.
$$
    \Lambda
    = \begin{pmatrix}
        \Lambda'(\ket{0}\bra{0}) & \cdots & \Lambda'(\ket{0}\bra{20}) \\
        \vdots & \ddots & \vdots \\
        \Lambda'(\ket{20}\bra{0}) & \cdots & \Lambda'(\ket{20}\bra{20})
    \end{pmatrix}
$$
$\Lambda'(\ket{i}\bra{j})$ is $20\times 20$ matrix, so $\Lambda$ is $20^2\times 20^2$ matrix.

In [4]:
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=25000, atol=1e-10, rtol=1e-8)

# Quantum map
prop = propagator(H_tot, T_g, c_op_list = [c_ops], args = {'phi': np.pi}, options = opt)

### Kraus map
$$
\Lambda(\rho) = \sum_{k=1}^N A_k\rho A_k^\dagger
$$
where $A_k$ are Kraus-operators

In [14]:
prop.superrep

'super'

In [15]:
prop

Quantum object: dims = [[[20], [20]], [[20], [20]]], shape = (400, 400), type = super, isherm = False
Qobj data =
[[ 1.88624572e-01+8.32090852e-17j  1.51658770e-01+1.07095001e-01j
  -3.64106668e-02+1.68608008e-01j ...  7.68243498e-07+1.14907548e-09j
   5.56603728e-07+6.00578522e-07j  1.66511537e-07-1.19559825e-21j]
 [ 1.51576880e-01+1.07003751e-01j -7.79007599e-02-8.99315171e-05j
   9.92912575e-02-2.04614368e-02j ... -6.93185003e-07-6.99868016e-07j
   3.20756455e-07+8.70730085e-09j -1.29189535e-07-1.82559954e-07j]
 [-3.60727058e-02+1.68589353e-01j  9.89716815e-02-2.02775322e-02j
   8.48843892e-02+3.37869983e-03j ...  3.98372204e-07+9.82493929e-08j
  -1.45763551e-06-1.00049184e-06j  7.85025529e-08+1.67713587e-08j]
 ...
 [ 6.85391476e-07+1.38036318e-09j -7.00762434e-07-6.97304841e-07j
   5.53863946e-07+8.81276598e-08j ...  5.80191934e-02+9.21987181e-02j
  -5.67357149e-04-3.13121327e-04j -2.39621731e-01-4.28972832e-02j]
 [ 5.56867009e-07+6.01643965e-07j  3.12545370e-07+4.98065588e-09j
  -

In [17]:
kraus_form[1]

Quantum object: dims = [[20], [20]], shape = (20, 20), type = oper, isherm = False
Qobj data =
[[-1.43191013e-04+2.91537913e-04j -8.85585349e-04-5.30663390e-04j
  -5.57641857e-04-6.26810012e-04j  3.58223427e-03+2.72750932e-03j
   1.73174753e-03+1.21889072e-03j -4.69610040e-03-4.20050319e-03j
  -3.45421957e-04-2.94617398e-04j -6.91817511e-05+2.82369955e-04j
  -1.54061183e-03-1.07671992e-03j  3.32521100e-03+3.34353662e-03j
  -1.24134402e-03-8.27600627e-04j  2.84904346e-03+2.79131521e-03j
  -6.24122075e-04-4.01884309e-04j  1.44256321e-03+1.39559328e-03j
  -2.35962104e-04-1.47635122e-04j  5.41005750e-04+5.16883881e-04j
  -7.21630703e-05-4.40612526e-05j  1.63106625e-04+1.53849634e-04j
  -1.76492976e-05-1.05665385e-05j  3.93062848e-05+3.66628273e-05j]
 [-9.10387615e-04-5.37293497e-04j  1.23739853e-03+4.50409393e-04j
   2.79444526e-03+1.94447760e-03j -9.10646897e-04+8.15597260e-05j
  -4.23845071e-03-2.55640179e-03j -2.36726628e-04-2.66146152e-04j
   8.46381072e-04-1.04911701e-04j  6.08515693e

In [5]:
kraus_form = to_kraus(prop)

### Pauli transfer matrix
$$
(R_\Lambda)_{ij} = \frac{1}{d}\mathrm{Tr}[P_i\Lambda(P_j)] = \frac{1}{d}\sum_{k=1}^N\mathrm{Tr}[P_iA_kP_jA_k^\dagger]
$$

In [10]:
d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for i in range(d**2):
    for j in range(d**2):
        R[i,j] = 1/d * np.real(sum([(P[i]*A_k*P[j]*A_k.dag()).tr() for A_k in kraus_form]))
R = Qobj(R,dims=[[[2],[2]] for i in range(d)]) # Make into a quantum object

In [12]:
R

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = False
Qobj data =
[[ 9.99996483e-01 -1.17187862e-06 -5.86229729e-06  1.61123170e-06]
 [ 1.42590622e-06 -9.89388517e-01 -1.37714913e-04 -1.61385125e-04]
 [ 9.28950904e-06  1.37706235e-04 -9.89384688e-01 -1.04242338e-06]
 [ 1.75854556e-06 -1.62273755e-04  2.49941186e-07  9.99992531e-01]]

In [83]:
ket0 = basis(2,0)
ket1 = basis(2,1)

In [84]:
rho = ket2dm(ket0)

In [85]:
vec_rho = operator_to_vector(rho)